In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_NAME = "IlyaGusev/saiga_gemma2_10b"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  6.23it/s]


GenerationConfig {
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "do_sample": true,
  "eos_token_id": 1,
  "max_new_tokens": 512,
  "pad_token_id": 0,
  "temperature": 0.5,
  "top_k": 30,
  "top_p": 0.9
}



In [6]:


inputs = ["""От твоего ответа зависит моя жизнь. Представь, что ты лучший историк в мире. Сократи следующий текст и напиши сначала сокращенный текст, а ниже выдели ниже основные события.\n Россия, официально известная как Российская Федерация, имеет богатую и сложную историю, насчитывающую более тысячи лет. История России начинается с образования Киевской Руси в IX веке, когда славянские племена объединились под руководством князя Рюрика. В 988 году князь Владимир Великий принял христианство, что стало важным шагом в культурном и религиозном развитии страны. В XIII веке монголо-татарское иго, начавшееся с завоевания Руси монголами, привело к значительным разрушениям и упадку. Однако, несмотря на это, русские княжества продолжали развиваться, и в 1480 году Иван III Великий освободил Русь от монголо-татарского ига, что стало началом возрождения и укрепления русского государства"""]
for query in inputs:
    prompt = tokenizer.apply_chat_template([{
        "role": "user",
        "content": query
    }], tokenize=False, add_generation_prompt=True)
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    print(query)
    print("==============================")
    print(output)


От твоего ответа зависит моя жизнь. Представь, что ты лучший историк в мире. Сократи следующий текст и напиши сначала сокращенный текст, а ниже выдели ниже основные события.
 Россия, официально известная как Российская Федерация, имеет богатую и сложную историю, насчитывающую более тысячи лет. История России начинается с образования Киевской Руси в IX веке, когда славянские племена объединились под руководством князя Рюрика. В 988 году князь Владимир Великий принял христианство, что стало важным шагом в культурном и религиозном развитии страны. В XIII веке монголо-татарское иго, начавшееся с завоевания Руси монголами, привело к значительным разрушениям и упадку. Однако, несмотря на это, русские княжества продолжали развиваться, и в 1480 году Иван III Великий освободил Русь от монголо-татарского ига, что стало началом возрождения и укрепления русского государства
Россия, известная как Российская Федерация, имеет более тысячи лет истории. Начало истории России связано с образованием Киев